# Travelling Salesperson Problem solved using genetic algorithms

In [1]:
# Bibliotecas e parametros
import numpy as np
import random
from datetime import datetime

n_cities = 20
n_population = 100
mutation_rate = 0.3

In [4]:
# Gerando uma lista de coordenadas representando cada cidade
coordinates_list = [[x,y] for x,y in zip(np.random.randint(0,100,n_cities),np.random.randint(0,100,n_cities))]
names_list = np.array(['Berlin', 'London', 'Moscow', 'Barcelona', 'Rome', 'Paris', 'Vienna', 'Munich', 'Istanbul', 'Kyiv', 'Bucharest', 'Minsk', 'Warsaw', 'Budapest', 'Milan', 'Prague', 'Sofia', 'Birmingham', 'Brussels', 'Amsterdam'])
cities_dict = { x:y for x,y in zip(names_list,coordinates_list)}

# Função para calcular a distância entre dois pontos
def compute_city_distance_coordinates(a,b):
    return ((a[0]-b[0])**2+(a[1]-b[1])**2)**0.5

def compute_city_distance_names(city_a, city_b, cities_dict):
    return compute_city_distance_coordinates(cities_dict[city_a], cities_dict[city_b])

cities_dict


{'Berlin': [48, 14],
 'London': [91, 0],
 'Moscow': [96, 97],
 'Barcelona': [80, 32],
 'Rome': [57, 50],
 'Paris': [79, 50],
 'Vienna': [38, 19],
 'Munich': [28, 82],
 'Istanbul': [36, 1],
 'Kyiv': [3, 94],
 'Bucharest': [25, 57],
 'Minsk': [9, 50],
 'Warsaw': [37, 31],
 'Budapest': [6, 99],
 'Milan': [96, 36],
 'Prague': [67, 49],
 'Sofia': [52, 58],
 'Birmingham': [35, 21],
 'Brussels': [40, 0],
 'Amsterdam': [91, 73]}

## 1. Crie o primeiro conjunto populacional
Embaralhamos aleatoriamente as cidades N vezes, onde N = tamanho_da_população.

In [5]:
# 1º conjunto populacional
def genesis(city_list, n_population):
    population_set = []
    for i in range(n_population):
        #Gerando aleatoriamente uma nova solução
        sol_i = city_list[np.random.choice(list(range(n_cities)), n_cities, replace=False)]
        population_set.append(sol_i)
    return np.array(population_set)

population_set = genesis(names_list, n_population)
population_set

array([['London', 'Bucharest', 'Paris', ..., 'Birmingham', 'Warsaw',
        'Kyiv'],
       ['Munich', 'Vienna', 'Paris', ..., 'Warsaw', 'Istanbul', 'Berlin'],
       ['Brussels', 'Kyiv', 'Vienna', ..., 'Paris', 'Munich',
        'Bucharest'],
       ...,
       ['London', 'Kyiv', 'Istanbul', ..., 'Barcelona', 'Milan',
        'Munich'],
       ['Vienna', 'London', 'Paris', ..., 'Minsk', 'Rome', 'Kyiv'],
       ['Istanbul', 'Milan', 'Rome', ..., 'Sofia', 'Paris', 'Vienna']],
      dtype='<U10')

## 2. Avalie a adequação das soluções
As soluções são definidas de forma que o primeiro elemento da lista seja a primeira cidade a visitar, depois a segunda, etc. e a última cidade esteja ligada à primeira.
A função de aptidão precisa calcular a distância entre as cidades subsequentes.

In [7]:
def fitness_eval(city_list, cities_dict):
    total = 0
    for i in range(n_cities-1):
        a = city_list[i]
        b = city_list[i+1]
        total += compute_city_distance_names(a,b, cities_dict)
    return total

def get_all_fitnes(population_set, cities_dict):
    fitnes_list = np.zeros(n_population)

    #Looping over all solutions computing the fitness for each solution
    for i in  range(n_population):
        fitnes_list[i] = fitness_eval(population_set[i], cities_dict)

    return fitnes_list

fitnes_list = get_all_fitnes(population_set,cities_dict)
fitnes_list

array([1098.37503584, 1049.23909243,  992.8812417 , 1149.36981691,
       1088.27483119, 1157.52594409,  973.62641731, 1058.40845816,
       1058.41612334, 1087.28981973, 1002.2951244 , 1136.28087521,
       1125.33351885, 1093.72477048, 1020.88712446, 1123.80870507,
        967.69767694, 1113.34056188, 1041.23213468, 1142.75116533,
       1141.34100837, 1226.79264372, 1174.214806  , 1041.25164813,
       1099.43389587,  968.76280429,  875.57965798, 1260.65472038,
       1093.24951322, 1152.27141811, 1144.87482877,  969.990546  ,
       1107.81692601, 1108.7191536 ,  833.7183617 , 1242.17952288,
       1055.94193477, 1039.61526371, 1117.22370693, 1015.92263347,
       1183.61817927, 1087.95575001,  914.46433492, 1095.5624872 ,
        844.82737864, 1216.90327418,  962.54027507, 1078.45892669,
       1048.51302628, 1199.78990112, 1126.36098257, 1025.05747055,
       1142.53999879, 1139.14186367, 1060.47614607,  925.4014766 ,
       1071.3119041 , 1144.94634573, 1152.30474267, 1062.47880

# 3. Seleção de progenitores
Selecionarei um novo conjunto de progenitores usando a Seleção da Roleta. Gera uma lista de pares de progenitores onde N= len(population_set) mas em cada posição existem duas soluções para mesclar

In [8]:
def progenitor_selection(population_set,fitnes_list):
    total_fit = fitnes_list.sum()
    prob_list = fitnes_list/total_fit

    #Notice there is the chance that a progenitor. mates with oneself
    progenitor_list_a = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)
    progenitor_list_b = np.random.choice(list(range(len(population_set))), len(population_set),p=prob_list, replace=True)

    progenitor_list_a = population_set[progenitor_list_a]
    progenitor_list_b = population_set[progenitor_list_b]


    return np.array([progenitor_list_a,progenitor_list_b])


progenitor_list = progenitor_selection(population_set,fitnes_list)
progenitor_list[0][2]

array(['Warsaw', 'Paris', 'Vienna', 'Berlin', 'Barcelona', 'Munich',
       'Sofia', 'Moscow', 'Bucharest', 'Minsk', 'Milan', 'Kyiv', 'Rome',
       'Budapest', 'Istanbul', 'Amsterdam', 'Brussels', 'Prague',
       'London', 'Birmingham'], dtype='<U10')

# 4. Crissamento
Para cada par de pais geraremos um par descendente. Como não podemos repetir cidades, o que faremos é copiar um pedaço aleatório de um progenitor e preencher os espaços em branco com o outro progenitor.

In [9]:
def mate_progenitors(prog_a, prog_b):
    offspring = prog_a[0:5]

    for city in prog_b:

        if not city in offspring:
            offspring = np.concatenate((offspring,[city]))

    return offspring



def mate_population(progenitor_list):
    new_population_set = []
    for i in range(progenitor_list.shape[1]):
        prog_a, prog_b = progenitor_list[0][i], progenitor_list[1][i]
        offspring = mate_progenitors(prog_a, prog_b)
        new_population_set.append(offspring)

    return new_population_set

new_population_set = mate_population(progenitor_list)
new_population_set[0]

array(['Warsaw', 'Rome', 'Minsk', 'Sofia', 'Milan', 'Brussels', 'Berlin',
       'Munich', 'Prague', 'London', 'Budapest', 'Vienna', 'Bucharest',
       'Moscow', 'Kyiv', 'Istanbul', 'Paris', 'Amsterdam', 'Barcelona',
       'Birmingham'], dtype='<U10')

# 5. Mutação
Agora, para cada elemento da nova população, adicionamos uma chance aleatória de troca

In [10]:
def mutate_offspring(offspring):
    for q in range(int(n_cities*mutation_rate)):
        a = np.random.randint(0,n_cities)
        b = np.random.randint(0,n_cities)

        offspring[a], offspring[b] = offspring[b], offspring[a]

    return offspring


def mutate_population(new_population_set):
    mutated_pop = []
    for offspring in new_population_set:
        mutated_pop.append(mutate_offspring(offspring))
    return mutated_pop

mutated_pop = mutate_population(new_population_set)
mutated_pop[0]

array(['Paris', 'Rome', 'Minsk', 'Moscow', 'Milan', 'Warsaw', 'Sofia',
       'Munich', 'Prague', 'London', 'Budapest', 'Vienna', 'Amsterdam',
       'Birmingham', 'Kyiv', 'Istanbul', 'Brussels', 'Bucharest',
       'Barcelona', 'Berlin'], dtype='<U10')

# 6. SParando
Para selecionar os critérios de parada, precisaremos criar um loop para parar primeiro. Então vou configurá-lo para fazer um loop em 1000 iterações.

In [11]:
best_solution = [-1,np.inf,np.array([])]
for i in range(10000):
    if i%100==0: print(i, fitnes_list.min(), fitnes_list.mean(), datetime.now().strftime("%d/%m/%y %H:%M"))
    fitnes_list = get_all_fitnes(mutated_pop,cities_dict)

    #Saving the best solution
    if fitnes_list.min() < best_solution[1]:
        best_solution[0] = i
        best_solution[1] = fitnes_list.min()
        best_solution[2] = np.array(mutated_pop)[fitnes_list.min() == fitnes_list]

    progenitor_list = progenitor_selection(population_set,fitnes_list)
    new_population_set = mate_population(progenitor_list)

    mutated_pop = mutate_population(new_population_set)

0 833.7183616991465 1076.6268060103646 06/05/24 20:31
100 795.7962266458421 1055.9789519611975 06/05/24 20:31
200 803.8137897708594 1066.4480481032815 06/05/24 20:31
300 861.9886729964207 1069.12850177717 06/05/24 20:31
400 855.5450649687737 1081.4305508203279 06/05/24 20:31
500 843.4783353725618 1044.4441562725647 06/05/24 20:31
600 841.0283891124741 1064.2333293166798 06/05/24 20:31
700 868.6940577349764 1077.4195816409667 06/05/24 20:31
800 860.7677811383339 1070.6768765812524 06/05/24 20:31
900 846.1874359398313 1074.1087137512384 06/05/24 20:32
1000 766.718465618718 1074.1430698084055 06/05/24 20:32
1100 825.0884276875576 1066.9122811201514 06/05/24 20:32
1200 898.4712522639837 1064.4239856426852 06/05/24 20:32
1300 746.1252999488288 1054.9003375471627 06/05/24 20:32
1400 811.2027269453176 1082.7118956124214 06/05/24 20:32
1500 806.9476321821105 1051.3917560504988 06/05/24 20:32
1600 866.7261233153187 1071.9054047819743 06/05/24 20:32
1700 817.5331757095483 1052.2898267820954 06/0

In [12]:
best_solution

[4245,
 606.8809197956663,
 array([['Moscow', 'Amsterdam', 'Vienna', 'Sofia', 'Munich', 'Kyiv',
         'Budapest', 'Prague', 'Warsaw', 'Minsk', 'Birmingham',
         'Istanbul', 'Brussels', 'Berlin', 'London', 'Milan', 'Barcelona',
         'Paris', 'Rome', 'Bucharest']], dtype='<U10')]